In [0]:
from time import time
from __future__ import division, print_function
import os
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, model_from_json, Sequential
from keras.layers import Input, Embedding, LSTM, Lambda, Softmax, Dense, Concatenate, Dropout, Add, add, concatenate
import keras.backend as K
from keras.optimizers import Adadelta, Adam
from keras.callbacks import ModelCheckpoint

In [0]:
from google.colab import drive
drive.mount('/content/drive')
TRAIN_CSV = '/content/drive/My Drive/Sem 3-2/NLA/Project/train.csv'
TEST_CSV = '/content/drive/My Drive/Sem 3-2/NLA/Project/test.csv'
EMBEDDING_FILE = '/content/drive/My Drive/Sem 3-2/NLA/Project/GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = '/content/drive/My Drive/Sem 3-2/NLA/Project/Models/LSTM/'

In [0]:
import nltk
nltk.download('stopwords')

In [0]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)

stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for index, row in train_df.iterrows():

    # Iterate through the text of both questions of the row
    for question in questions_cols:

        q2n = []  # q2n -> question numbers representation
        for word in text_to_word_list(row[question]):

            # Check for unwanted words
            if word in stops and word not in word2vec.vocab:
                continue

            if word not in vocabulary:
                vocabulary[word] = len(inverse_vocabulary)
                q2n.append(len(inverse_vocabulary))
                inverse_vocabulary.append(word)
            else:
                q2n.append(vocabulary[word])

        # Replace questions as word to question as number representation
        train_df.at[index, question] = q2n
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [0]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max())

# Split to train validation test
test_size = 40000
validation_size = 40000
training_size = len(train_df) - validation_size - test_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values
Y_test = Y_test.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [0]:
# Model variables
n_hidden = 64
gradient_clipping_norm = 1.25
batch_size = 2048
n_epoch = 20

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Softmax classification
added = Add()([left_output, right_output])
softmax_distance = Dense(2, activation='softmax')(added)

# Pack it all up into a model
lstm = Model([left_input, right_input], [pred])

# Adam optimizer, with learning rate
optimizer = Adam(learning_rate=0.001)

lstm.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

lstm_trained = lstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [0]:
# serialize model to JSON
model_json = lstm.to_json()
with open(MODEL_SAVING_DIR + "batch2048.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
lstm.save_weights(MODEL_SAVING_DIR + "batch2048.h5")

In [0]:
loss, acc = lstm.evaluate([X_test['left'], X_test['right']], Y_test)
print("Loss =", loss, "Accuracy =", acc)

In [0]:
# load json and create model
json_file = open(MODEL_SAVING_DIR + "batch2048.json", 'rb')
loaded_model_json = json_file.read()
json_file.close()
lstm = model_from_json(loaded_model_json)

# load weights into new model
lstm.load_weights(MODEL_SAVING_DIR + "batch2048.h5")

results = lstm.evaluate([X_test['left'], X_test['right']], Y_test)

In [0]:
# Plot accuracy
plt.plot(lstm_trained.history['accuracy'])
plt.plot(lstm_trained.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(lstm_trained.history['loss'])
plt.plot(lstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
